In [1]:

!pip install pandas --quiet


## *Reading Time Series Data from Files*, 

This notebook is an introduction to time series data. This file shows you how to read data from various and commonly used file
types, whether stored locally or on the cloud. The techniques will highlight advanced options
for ingesting, preparing, and transforming data into a time series DataFrame for
later analysis.  

We will cover the following Methods on how to ingest data into a pandas DataFrame:
* Reading data from CSVs and other delimited files
* Reading data from an Excel file
* Reading data from URLs


# Method 1: Reading from CSVs and Other Delimited Files
In this Method, you will use the `pandas.read_csv()` function, which offers a large set
of parameters that you will explore to ensure the data is properly read into a time series
DataFrame. In addition, you will learn how to specify an index column, parse the index
to be of the type `DatetimeIndex`, and parse string columns that contain dates into
datetime objects.

[Official Documentation of pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

In [2]:
import pandas as pd
from pathlib import Path
pd.__version__

'2.0.0'

In [4]:
filepath = Path('./movieboxoffice.csv')

# the 'movieboxoffice.csv' should be in the same directory as this notebook file for the above to work, else specify the correct path as per your directory structure

In [7]:
ts = pd.read_csv(filepath, header=0, parse_dates=['Date'], index_col=0, usecols=['Date','DOW','Daily','Forecast','Percent Diff'])
ts.head()

C:\Users\mailt\AppData\Local\Temp\ipykernel_6276\1783708044.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.read_csv(filepath, header=0, parse_dates=['Date'], index_col=0, usecols=['Date','DOW','Daily','Forecast','Percent Diff'])


,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


The first column in the CSV file contains movie release dates, and it needs to
be set as an index of type `DatetimeIndex` (`index_col=0` and `parse_
dates=['Date']`). Specify which columns you want to include by providing a list
of column names to `usecols`. The default behavior is that the first row includes the
header (`header=0`).

Print a summary of the `DataFrame` to check the index and column data types:

In [12]:
ts.info()

# notice the dtype of Daily and Forecast column entries. They should be numeric and not object, right?

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOW           128 non-null    object 
 1   Daily         128 non-null    float64
 2   Forecast      128 non-null    float64
 3   Percent Diff  128 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


In [9]:
ts.describe()

,DOW,Daily,Forecast,Percent Diff
count,128,128,128,128
unique,7,123,124,128
top,Friday,$0.00,$0.00,-46.48%
freq,19,5,4,1


## **Data Cleanup**
* Notice that the `Date` column is now an `index` (not a `column`) of type
`DatetimeIndex`. Additionally, both Daily and Forecast columns have the
wrong `dtype inference`. 
* You would expect them to be of type `float`. The issue is
due to the source CSV file containing dollar signs ($) and thousand separators (,)
in both columns.
* To fix this, you need to remove both the dollar sign ($) and thousand separators (,)
or any non-numeric character. You can accomplish this using `str.replace()`,
which can take a regular expression to remove all non-numeric characters but
exclude the period (.) for the decimal place. 
Removing these characters does not convert the `dtype`, so you will need to "cast" those two columns as a `float dtype`
using `.astype(float)`

In [10]:
clean = lambda x: x.str.replace('[^\d]', '', regex=True)                                                      
c_df = ts[['Daily', 'Forecast']].apply(clean, axis=1)
ts[['Daily', 'Forecast']] = c_df.astype(float)

In [11]:
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,12578989.0,23503646.0,-46.48%
2021-04-27,Saturday,9937401.0,19762255.0,-49.72%
2021-04-28,Sunday,8220316.0,11699126.0,-29.74%
2021-04-29,Monday,3353026.0,6665265.0,-49.69%
2021-04-30,Tuesday,3010524.0,3482819.0,-13.56%


In [13]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOW           128 non-null    object 
 1   Daily         128 non-null    float64
 2   Forecast      128 non-null    float64
 3   Percent Diff  128 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


In [16]:
ts.describe()

,Daily,Forecast
count,1.280000e+02,1.280000e+02
mean,5.876124e+05,9.366103e+05
std,1.755046e+06,3.082168e+06
min,0.000000e+00,0.000000e+00
25%,5.636750e+03,9.266750e+03
50%,4.266750e+04,6.523100e+04
75%,1.927085e+05,2.893818e+05
max,1.257899e+07,2.350365e+07


To get the exact memory usage for each column, including the index, you can
use the memory_usage() method:

In [17]:
ts.memory_usage()

# ts.memory_usage().sum()

Index           1024
DOW             1024
Daily           1024
Forecast        1024
Percent Diff    1024
dtype: int64

So far, you have used a few of the available parameters when reading a CSV file using
read_csv.

## Using `date_parser`

In [18]:
ts.dtypes

DOW              object
Daily           float64
Forecast        float64
Percent Diff     object
dtype: object

You can also pass a lambda function that uses the `to_datetime` function in
pandas to `date_parser`. You can specify the string representation for the date format
inside `to_datetime()`, as demonstrated in the following code:

In [21]:
date_parser = lambda x: pd.to_datetime(x, format="%d-%b-%y")
ts = pd.read_csv(filepath,
                 parse_dates=[0],
                 index_col=0,
                 date_parser=date_parser,
                 usecols=[0,1,3, 7, 6])
ts.head()

C:\Users\mailt\AppData\Local\Temp\ipykernel_6276\4043552452.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ts = pd.read_csv(filepath,


,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


The preceding code will print out the first five rows of the ts DataFrame displaying
a properly parsed Date index.

Let's break it down. In the preceding code, you passed two arguments to the `to_datetime` function: the object to convert to datetime and an `explicit format string`.
Since the date is stored as a string in the form `26-Apr-21`, you passed `"%d-%b-%y"` to
reflect that:
* %d represents the day of the month, such as 01 or 02.
* %b represents the abbreviated month name, such as Apr or May.
* %y represents a two-digit year, such as 19 or 20.

# Method 2: Reading data from an Excel file
In this Method, you will be using the `pandas.read_excel()` function and examining 
the various parameters available to ensure the data is read properly as a `DataFrame` with 
a `DatetimeIndex` for time series analysis.

In addition, you will explore different optionsto read Excel files with multiple sheets, install `openpyxl` using conda or pip, run the following command in the terminal:
```
conda install openpyxl
```
or
```
pip install openpyxl
```

In [29]:
#!conda install openpyxl -y
!pip install openpyxl --quiet

import warnings
warnings.simplefilter("ignore")

Import the libraries for this Method
Read the Excel (.xlxs) file using the `read_excel()` function. 

In [23]:
import pandas as pd
from pathlib import Path
filepath = Path('./boxoffice_by_month.xlsx')

In [24]:
import openpyxl
openpyxl.__version__

'3.0.10'

In [25]:
excelfile = pd.ExcelFile(filepath)
excelfile.sheet_names

['April', 'August', 'July', 'June', 'May']

In [26]:
excelfile.parse('July')

,Date,Daily,Month
0,2021-07-01,$578.32,July
1,2021-07-02,$386.08,July
2,2021-07-03,$409.16,July
3,2021-07-04,$512.57,July
4,2021-07-05,$693.22,July
5,2021-07-06,$777.98,July
6,2021-07-07,$525.73,July
7,2021-07-08,$239.25,July
8,2021-07-09,$316.44,July
9,2021-07-10,$234.88,July


* The box_office data will be partitioned (split) by month, where each sheet contained sales for 
a particular month. 
* In this case, concatenating the two DataFrames is a natural choice. The 
`pandas.concat()` function is like the `DataFrame.append()` function, in which the second DataFrame was added (appended) to the end of the first DataFrame.

In [40]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=0,
                    sheet_name=[0,1],
                    parse_dates=True)
ts.keys()

dict_keys([0, 1])

In [41]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=0,
                    sheet_name=['June','May'],
                    parse_dates=True)
ts.keys()

dict_keys(['June', 'May'])

In [42]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=0,
                    sheet_name=None,
                    parse_dates=True)
ts.keys()

dict_keys(['April', 'August', 'July', 'June', 'May'])

In [43]:
ts['June'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30 entries, 2021-06-01 to 2021-06-30
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Daily   30 non-null     object
 1   Month   30 non-null     object
dtypes: object(2)
memory usage: 720.0+ bytes


In [44]:
ts['May'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2021-05-01 to 2021-05-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Daily   31 non-null     object
 1   Month   31 non-null     object
dtypes: object(2)
memory usage: 744.0+ bytes


In [36]:
ts_combined = pd.concat([ts['June'],ts['May']])

In [37]:
ts_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries,  $2,955.72  to  $1,819.52 
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    61 non-null     datetime64[ns]
 1   Month   61 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1.4+ KB


In [45]:
pd.concat(ts).index

MultiIndex([( 'April', '2021-04-26'),
            ( 'April', '2021-04-27'),
            ( 'April', '2021-04-28'),
            ( 'April', '2021-04-29'),
            ( 'April', '2021-04-30'),
            ('August', '2021-08-01'),
            ('August', '2021-08-02'),
            ('August', '2021-08-03'),
            ('August', '2021-08-04'),
            ('August', '2021-08-05'),
            ...
            (   'May', '2021-05-22'),
            (   'May', '2021-05-23'),
            (   'May', '2021-05-24'),
            (   'May', '2021-05-25'),
            (   'May', '2021-05-26'),
            (   'May', '2021-05-27'),
            (   'May', '2021-05-28'),
            (   'May', '2021-05-29'),
            (   'May', '2021-05-30'),
            (   'May', '2021-05-31')],
           names=[None, 'Date'], length=128)

To reduce the number of levels, you can use the `droplevel(level=0)` method 
to drop the first level after `pandas.concat()` shown as follows:

In [46]:
ts_combined = pd.concat(ts).droplevel(level=0)
ts_combined.head()

,Daily,Month
Date,,
2021-04-26,"$125,789.89",April
2021-04-27,"$99,374.01",April
2021-04-28,"$82,203.16",April
2021-04-29,"$33,530.26",April
2021-04-30,"$30,105.24",April


In [48]:
ts = pd.read_excel(filepath,
                   index_col=0,
                   sheet_name='June',
                   parse_dates=True)
type(ts)

pandas.core.frame.DataFrame

## There is more
* Using `ExcelFile`

In [50]:
excelfile = pd.ExcelFile(filepath)
excelfile.parse(sheet_name='June',
                index_col=0,
                parse_dates=True).head()


,Daily,Month
Date,,
2021-06-01,"$2,955.72",June
2021-06-02,"$2,249.78",June
2021-06-03,$753.48,June
2021-06-04,"$1,044.94",June
2021-06-05,$771.17,June


# Method 3: Reading data from a URL
* Many of the pandas reader functions can read data from remote locations by passing 
a URL path. 
* For example, both `read_csv()` and `read_excel()` can take a URL to 
read a file that is accessible via the internet. In this Method, you will read a `CSV` file using 
`pandas.read_csv()` and `Excel` files using `pandas.read_excel()` from remote 
locations, such as `GitHub`. 
* You will also read data directly from an HTML page into a pandas DataFrame

In [51]:
import pandas as pd
import  lxml
print(f'''
pandas -> {pd.__version__}
lxml -> {lxml.__version__}
''')


pandas -> 2.0.0
lxml -> 4.9.1



## Reading from GitHub
* This is most important part since you'll be using it frequently in your project to read the data from the project's github page
* To read a CSV file from GitHub, you will need the URL to the `raw content`. 
* If you copy the file's GitHub URL from the browser and use it as the file path, you will get 
a URL that looks like this: https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv  
* This URL is a pointer to the web page in GitHub and 
not the data itself; hence when using `pd.read_csv()`, it will throw an error

In [55]:
# example of produced error
url = 'https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv'
# pd.read_csv(url)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 28, saw 367


Instead, you will need the raw content, which will give you a URL that 
looks like this:
The first column in the file is the Date column. You will need to parse (parse_
date parameter) and convert it to DatetimeIndex (index_col parameter).

In [58]:
url = 'https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv'

raw_url = 'https://raw.githubusercontent.com/FuTSA23/time-series-analysis-datasets/main/AirQualityUCI.csv'
# date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%Y")

df = pd.read_csv(raw_url)

URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>

## Reading from HTML
pandas offers an elegant way to read HTML tables and convert the content into a pandas 
DataFrame using the pandas.read_html() function: 
* In the following Method, we will extract HTML tables from Wikipedia for COVID-19 
pandemic tracking cases by country and by territory (https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory)

In [59]:
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
results = pd.read_html(url)
print(len(results))

68


In [61]:
# for i, k in enumerate(results):
#     print(i)
#     display(k.head())

In [62]:
df = results[15]
df.columns

Index(['Region[29]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[30]'],
      dtype='object')

In [63]:
df[['Total cases', 'Total deaths', 'Cases per million']].head()

,Total cases,Total deaths,Cases per million
0,179537758,1185108,401363
1,103783777,1133607,281404
2,57721948,498259,247054
3,65835789,1313061,153151
4,25646533,434988,108307


## Example how `read_html()` works

In [64]:
import pandas as pd
html = """
 <table>
   <tr>
     <th>Ticker</th>
     <th>Price</th>
   </tr>
   <tr>
     <td>MSFT</td>
     <td>230</td>
   </tr>
   <tr>
     <td>APPL</td>
     <td>300</td>
   </tr>
     <tr>
     <td>MSTR</td>
     <td>120</td>
   </tr>
 </table>

 </body>
 </html>
 """

df = pd.read_html(html)
df[0]


,Ticker,Price
0,MSFT,230
1,APPL,300
2,MSTR,120


In the preceding code, the `read_html()` function parsed the HTML code and converted 
the HTML table into a `pandas DataFrame`. The headers between the `<th>` and `</th>`
tags represent the `column names` of the DataFrame, and the content between the `<tr></td>` and `</td></tr>` tags represent the row data of the DataFrame. 

## Using `attr` option in `read.html()`
The `read_html()` function has an optional `attr` argument, which takes a dictionary 
of `valid HTML <table> attributes`, such as `id or class`. For example, you can use 
the `attr` parameter to narrow down the tables returned to those that match the class
attribute sortable as in `<table class="sortable">`. The `read_html` function will inspect the entire HTML page to ensure you target the right set of attributes.

In [65]:
#!conda install html5lib beautifulSoup4
!pip install html5lib beautifulSoup4 --quiet

In [66]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
df = pd.read_html(url, attrs={'class': 'sortable'})
len(df)

5

In [67]:
df[3].columns

Index(['Region[29]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[30]'],
      dtype='object')